In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from scipy.stats import ttest_1samp

In [2]:
processed_dataset = pd.read_csv('data/processed-data.csv')
processed_dataset.head()

,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target,y_labels
0,0.284211,0.340000,0.056604,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.00,0.000000,0.0,0.372093,0.488889,0.766182,Dropout,0.0
1,0.684211,0.500000,0.037736,0.0,0.230769,0.133333,0.230769,0.741722,0.0,0.0,0.26087,0.181818,0.30,0.735897,0.0,0.732558,0.111111,0.640687,Graduate,1.0
2,0.284211,0.313684,0.037736,0.0,0.230769,0.000000,0.000000,0.000000,0.0,0.0,0.26087,0.000000,0.00,0.000000,0.0,0.372093,0.488889,0.766182,Dropout,0.0
3,0.284211,0.258947,0.056604,0.0,0.230769,0.177778,0.230769,0.711447,0.0,0.0,0.26087,0.303030,0.25,0.667692,0.0,0.209302,0.000000,0.124174,Graduate,1.0
4,0.052632,0.489474,0.528302,0.0,0.230769,0.200000,0.192308,0.653422,0.0,0.0,0.26087,0.181818,0.30,0.700000,0.0,0.732558,0.111111,0.640687,Graduate,1.0


In [117]:
feature_columns = processed_dataset.columns.drop(['Target', 'y_labels'])
X = processed_dataset[feature_columns].to_numpy(dtype=float)
y = processed_dataset['y_labels'].to_numpy(dtype=int)

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

In [ ]:
# Augment X matrices with column of 1s (for intercept)
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])

In [ ]:
# Train a soft-margin linear SVM via SGD using hinge loss and slack variable C
def train_svm_sgd(X, y, epochs=1000, lr=0.01, C=1.0):
    n_samples, n_features = X.shape
    w = np.zeros(n_features) 
    
    for _ in range(epochs):
        for i in range(n_samples):
            x_i, y_i = X[i], y[i]
            margin = y_i * (w.dot(x_i))
            if margin >= 1:
                grad_w = w
            else:
                grad_w = w - C * y_i * x_i
            w -= lr * grad_w
    return w

In [121]:
# Testing model on testing set
w_final = train_svm_sgd(X_train, y_train, epochs=1000, lr=0.01, C=1.0)

In [ ]:
coeffs    = w_final[:-1]
intercept = w_final[-1]

print(f"Intercept: {intercept:.4f}\n")
print("Feature Coefficients:")
for name, coeff in zip(feature_columns, coeffs):
    print(f"- {name:50s} {coeff: .4f}")

Intercept: 0.3286

Feature Coefficients:
- Previous qualification (grade)                      0.1643
- Admission grade                                     0.1643
- Age at enrollment                                   0.1643
- Curricular units 1st sem (credited)                 0.1643
- Curricular units 1st sem (enrolled)                 0.1643
- Curricular units 1st sem (evaluations)              0.1643
- Curricular units 1st sem (approved)                 0.1643
- Curricular units 1st sem (grade)                    0.1643
- Curricular units 1st sem (without evaluations)      0.1643
- Curricular units 2nd sem (credited)                 0.1643
- Curricular units 2nd sem (enrolled)                 0.1643
- Curricular units 2nd sem (evaluations)              0.1643
- Curricular units 2nd sem (approved)                 0.1643
- Curricular units 2nd sem (grade)                    0.1643
- Curricular units 2nd sem (without evaluations)      0.1643
- Unemployment rate                         

In [123]:
# Test Accuracy
test_preds = np.sign(X_test.dot(w_final))
test_acc = (test_preds == y_test).mean()
print(f"\nTest Accuracy: {test_acc:.3f}")


Test Accuracy: 0.679


Because the linear SVM produces unsatisfactory test accuracy, we’ll switch to the dual-optimization (kernel) SVM. To ensure our feature set is robust, we’ll first run five-fold cross-validation on the linear SVM using fixed folds each time in order to evaluate each coefficient via statistical analysis. Features that fail to reach some threshold (i.e. features with p-value >= 0.05) will be removed. We expect that this will improve the performance of the subsequent kernel-based SVM.

In [124]:
kf = KFold(n_splits=5, shuffle=False) # shuffle=False ensures folds remain fixed across iterations
coefs, val_accs = [], []

In [125]:
for tr_idx, val_idx in kf.split(X_train):
    X_tr, y_tr = X_train[tr_idx], y_train[tr_idx]
    X_val, y_val = X_train[val_idx], y_train[val_idx]
    
    w_fold = train_svm_sgd(
        X_tr, y_tr,
        epochs=500,
        lr=0.01,
        C=1.0
    )
    coefs.append(w_fold)
    
    preds = np.sign(X_val.dot(w_fold))
    val_accs.append((preds == y_val).mean())

coefs = np.vstack(coefs)

In [ ]:
print("Validation Accuracies:", val_accs)
print(f"Mean Cross-Validation Accuracy: {np.mean(val_accs):.3f} ± {np.std(val_accs, ddof=1):.3f}")

Validation Accuracies: [0.6624293785310734, 0.652542372881356, 0.7076271186440678, 0.7033898305084746, 0.6676096181046676]
Mean Cross-Validation Accuracy: 0.679 ± 0.025


Since the 5-fold cross-validation accuracy is very similar to the test accuracy, we will proceed with statistical analysis to identify the most significant features in an effort to improve the performance of the kernel-based SVM.

In [132]:
# Statistical Analysis on 5-fold Cross Validation
coef_mean = coefs.mean(axis=0)
coef_std  = coefs.std(axis=0, ddof=1)
t_stats, p_vals = ttest_1samp(coefs, popmean=0.0, axis=0)

results = pd.DataFrame({
    'feature': list(feature_columns) + ['bias'],
    'mean_w': coef_mean,
    'std_w' : coef_std,
    't_stat': t_stats,
    'p_value': p_vals
}).sort_values('p_value')

results

,feature,mean_w,std_w,t_stat,p_value
7,Curricular units 1st sem (grade),0.192433,0.000414,1038.176761,5.164917e-12
13,Curricular units 2nd sem (grade),0.191949,0.001945,220.652429,2.530795e-09
11,Curricular units 2nd sem (evaluations),0.073818,0.000875,188.741056,4.727205e-09
16,Inflation rate,0.133085,0.001759,169.174932,7.323287e-09
0,Previous qualification (grade),0.122619,0.002372,115.593234,3.358961e-08
6,Curricular units 1st sem (approved),0.057048,0.001158,110.130696,4.076421e-08
12,Curricular units 2nd sem (approved),0.071900,0.001482,108.454117,4.334320e-08
15,Unemployment rate,0.128209,0.003012,95.179553,7.305620e-08
18,bias,0.332162,0.007878,94.279968,7.588359e-08
1,Admission grade,0.103068,0.002747,83.911491,1.209078e-07


All features with a p-value less than 0.05 are statistically significant and will be kept. Features with higher p-values will be excluded from the model.

Significant Features (p < 0.05):
- Curricular units 1st sem (grade)
- Curricular units 2nd sem (grade)
- Curricular units 2nd sem (evaluations)
- Inflation rate
- Previous qualification (grade)
- Curricular units 1st sem (approved)
- Curricular units 2nd sem (approved)
- Unemployment rate
- Admission grade
- Curricular units 1st sem (evaluations)
- Curricular units 2nd sem (enrolled)
- Curricular units 1st sem (without evaluations)
- Age at enrollment
- Curricular units 1st sem (enrolled)
- GDP

Non-Significant Features (p ≥ 0.05):
- Curricular units 2nd sem (without evaluations)
- Curricular units 2nd sem (credited)
- Curricular units 1st sem (credited)

In [134]:
# Removing non-significant features
nonsignificant_features = ['Curricular units 2nd sem (without evaluations)', 'Curricular units 2nd sem (credited)','Curricular units 1st sem (credited)']

processed_dataset = processed_dataset.drop(columns=nonsignificant_features, errors='ignore', axis=1)
processed_dataset.head()

,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Unemployment rate,Inflation rate,GDP,Target,y_labels
0,0.284211,0.340000,0.056604,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.00,0.000000,0.372093,0.488889,0.766182,Dropout,0.0
1,0.684211,0.500000,0.037736,0.230769,0.133333,0.230769,0.741722,0.0,0.26087,0.181818,0.30,0.735897,0.732558,0.111111,0.640687,Graduate,1.0
2,0.284211,0.313684,0.037736,0.230769,0.000000,0.000000,0.000000,0.0,0.26087,0.000000,0.00,0.000000,0.372093,0.488889,0.766182,Dropout,0.0
3,0.284211,0.258947,0.056604,0.230769,0.177778,0.230769,0.711447,0.0,0.26087,0.303030,0.25,0.667692,0.209302,0.000000,0.124174,Graduate,1.0
4,0.052632,0.489474,0.528302,0.230769,0.200000,0.192308,0.653422,0.0,0.26087,0.181818,0.30,0.700000,0.732558,0.111111,0.640687,Graduate,1.0


In [ ]:
processed_dataset.to_csv('data/significant-features-data.csv', index=False)